# Colab MSA-Search with Many Proteins

Runs MSA-Search NIM and saves result as `A3M` format file.

MSA-Search NIM: https://build.nvidia.com/colabfold/msa-search

27Aug2025

## 1.1 Set Up the Environment

In [ ]:
!pip install pandas tqdm httpx "fastapi[standard]"

In [ ]:
import os, requests, re
import shutil
import pandas as pd

from pathlib import Path
from tqdm import tqdm

from google.colab import userdata, files

### Define Input File and Output Directory

In [ ]:
csv_file_path = "UniprotID_and_FastaSequences_100_Examples.csv"

output_dir = "/content/output"

## 1.2 Set Up `output` Directory and `API_KEY`

**NOTE:** Be sure to follow the steps in the README to embed your NVIDIA `API_KEY` into your Google Colab environment.

In [ ]:
API_KEY = userdata.get('API_KEY')

# Ensure output directory exists, create if not present
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

## 1.3 Define Functions

In [ ]:
MSA_DATABASES = ['Uniref30_2302', 'colabfold_envdb_202108', 'PDB70_220313']

def msa_search(sequence, API_KEY, databases=MSA_DATABASES):
    msa_search_url = "https://health.api.nvidia.com/v1/biology/colabfold/msa-search/predict"
    payload = {
        "sequence": sequence,
        "databases": databases,
        "e_value": 0.0001,
        "iterations": 1,
        "max_msa_sequences": 10000,
        "run_structural_template_search": False,
        "output_alignment_formats": ["a3m"],
    }
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "content-type": "application/json",
        "NVCF-POLL-SECONDS": "300",
    }
    # Call MSA-Search NIM
    response = requests.post(msa_search_url, json=payload, headers=headers)
    return response.json()


def parse_sequences(input_string, n, sequence):
    """
    Parse the output of alignments from the MSA-Search NIM to be used downstream
    
    Args:
        input_string (str): The output file of alignments in a string format
        n (int): The amount of alignments to return from the output when parsing
        sequence (str): The query sequence for alignment
    
    Returns:
        list: A list of alignment identifiers and sequences, starting with the query,
              where the amount of sequences is given by n
    """
    # Output is parsed to have a line for the sequence id and sequence itself so `n` returns correlates to n*2 lines
    n = n * 2
    # First, handle the `Query` block separately
    lines = input_string.strip().split('\n')
    # Now process the rest of the lines
    remaining_string = "\n".join(lines[:])
    # Regex to find blocks starting with `>` and then followed by a sequence.
    pattern = re.compile(r'\n>(.*?)\n(.*?)(?=\n>|\Z)', re.DOTALL)
    matches = pattern.finditer(remaining_string)
    output_list_to_order = []
    for match in matches:
        # The name is the first capturing group, split by tab and take the first part
        name_full = match.group(1).split('\t')[0]
        SW_score = match.group(1).split('\t')[1]
        # The sequence is the second capturing group
        sequence_raw = match.group(2).strip()
        sequence = ''.join(char for char in sequence_raw if char.isupper() or not char.isalpha())
        # Store the aligned sequence in the list of outputs by name, sequence, Smith-Waterman score
        output_list_to_order.append((f'>{name_full}', sequence, int(SW_score)))
    output_lines = output_list_to_order[:n]
    return output_lines


def write_alignments_to_a3m(alignments_data, uniprot_id, output_dir):
    """
    Write alignment data to a3M format file.
    
    Args:
        alignments_data: Either a list of alternating headers/sequences or a string containing alignments
        uniprot_id (str): Uniprot ID of the protein
        output_dir (str): Directory for the output a3M file
    
    Returns:
        str: Path to the created a3M file
    """
    output_path = Path(output_dir) / f"{uniprot_id}_msa_alignments.a3m"
    # Handle both list and string input formats
    if isinstance(alignments_data, list):
        alignments_string = '\n'.join(alignments_data)
    elif isinstance(alignments_data, str):
        alignments_string = alignments_data
    else:
        raise ValueError("alignments_data must be either a list or string")
    # Count sequences for reporting
    sequence_count = alignments_string.count('>')
    print(f"Writing {sequence_count} sequences to a3M format: {output_path}")
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            # Write the alignments
            f.write(alignments_string)
            # Ensure file ends with newline
            if not alignments_string.endswith('\n'):
                f.write('\n')
        # Verify the file was created successfully
        if output_path.exists():
            file_size = output_path.stat().st_size
            print(f"Successfully created a3M file:")
            print(f"File: {output_path}")
            print(f"Size: {file_size:,} bytes")
            print(f"Sequences: {sequence_count}")
            return str(output_path)
            files.download(output_path)
        else:
            raise IOError(f"Failed to create file {output_path}")
    except Exception as e:
        print(f"Error writing a3M file: {e}")
        raise



def process_msa_alignments(msa_response_dict, sequence, uniprot_id, output_dir, databases=MSA_DATABASES, max_sequences_per_db=10000):
    """
    Process MSA alignments from multiple databases and merge them into A3M format.
    
    Args:
        msa_response_dict (dict): MSA response data containing alignments
        sequence (str): Query sequence for alignment
        uniprot_id (str): Uniprot ID of the protein
        output_dir (str): Output directory for the A3M file
        databases (list): List of database names to process
        max_sequences_per_db (int): Maximum number of sequences to parse per database
    
    Returns:
        str: Path to the created A3M file
    """
    all_parsed_dataset_output = []
    for database in databases:
        print(f"Parsing results from database: {database}")
        # Pull string of alignments stored in json output for specific dataset
        a3m_dict_msa_search = msa_response_dict['alignments'][database]['a3m']['alignment']
        a3m_dict_msa_search_parsed = parse_sequences(a3m_dict_msa_search, max_sequences_per_db, sequence)
        num_sequences_aligned = (len(a3m_dict_msa_search_parsed))
        print(f"Number of sequences aligned: {num_sequences_aligned}")
        all_parsed_dataset_output.extend(a3m_dict_msa_search_parsed)
    # Sort all the alignments based off of the alignment score
    all_parsed_dataset_output.sort(key=lambda x: x[2], reverse=True)
    # Now that the alignments across all datasets are sorted, reformat each entry to name and sequence
    sorted_parsed_output_formatted = []
    for align_tuple in all_parsed_dataset_output:
        sorted_parsed_output_formatted.append(align_tuple[0])
        sorted_parsed_output_formatted.append(align_tuple[1])
    merged_alignments_protein = [f">query_sequence\n{sequence}"]
    merged_alignments_protein.extend(sorted_parsed_output_formatted)
    print(f"Total merged alignments: {len(merged_alignments_protein)}")
    # Write merged_alignments_protein to a3M format
    a3m_file_path = write_alignments_to_a3m(
        merged_alignments_protein, 
        uniprot_id,
        output_dir
    )
    return a3m_file_path

## 1.3 Load File

In [ ]:
df = pd.read_csv(csv_file_path, low_memory=False)
print(df.shape)
df.head()

In [ ]:
df.columns

## 1.4 Parse Protein Names and Sequences to only unique pairs

In [ ]:
seq_uniprot = df['uniprot_id'].tolist()
sequences = df['fasta_uniprot_seq'].tolist()
sequences = tuple(zip(seq_uniprot, sequences))
sequences = sorted(list(set(sequences)))
print(f"Number of unique sequences: {len(sequences)} from parent dataset of {df.shape[0]} sequences")
sequences[0]

## 1.5 Loop Through Protein Names and Sequences with MSA-Search NIM

In [ ]:
for seq_id, seq in tqdm(sequences[:1]):
    msa_response_dict = msa_search(seq, API_KEY)
    process_msa_alignments(msa_response_dict, seq, seq_id, output_dir)